Though DVI has lower reconstruction error, but higher top1-top2 difference. Here we focus more on important dimensions,
hope to fix this problem

In [1]:
# import modules
from deepvisualinsight.MMS import MMS
from deepvisualinsight import utils
import sys
import os
import numpy as np
import time
import torch
import json
import tensorflow as tf
import time
from scipy.special import softmax
from deepvisualinsight.backend import get_alpha

In [2]:
content_path = "E:\\DVI_exp_data\\RQ1\\fix_border"
content_path = "E:\\DVI_exp_data\\resnet18_mnist"
sys.path.append(content_path)

from Model.model import *
net = resnet18()
classes = ("airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

mms = MMS(content_path, net, 100, 100, 1, 512, 10, classes, cmap="tab10", resolution=400, neurons=256,
           verbose=1, temporal=False, split=-1, advance_border_gen=True, attack_device="cuda:0")

Finish loading content!


In [3]:
# hyperparameters
EPOCH = 10

In [4]:
repr_model = torch.nn.Sequential(*(list(mms.model.children())[-1:]))
repr_model = repr_model.to(mms.device)
repr_model = repr_model.eval()

In [5]:
for param in repr_model.parameters():
    param.requires_grad = False

In [6]:
train_data = mms.get_epoch_train_repr_data(EPOCH)
border_points = mms.get_epoch_border_centers(EPOCH)
fitting_data = np.concatenate((train_data, border_points), axis=0)

In [8]:
alpha = get_alpha(repr_model, fitting_data, temperature=.01, device=torch.device("cuda:0"), verbose=1)

Gradients calculation: 66.00 seconds	softmax with temperature: 1.00 seconds


In [ ]:
mms.prepare_visualization_for_all(alpha)